# The interesting topics in this year

In [1]:
from semanticscholar import SemanticScholar
import time
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

def get_paper_citation(title):
    sch = SemanticScholar()
    time.sleep(1)
    title = title.replace('-', ' ')
    try:
        papers = sch.search_paper(title)
        if len(papers) > 0:
            return papers[0].citationCount
    except:
        print(f'Error in get citation for {title}')
    
    return 0

In [2]:
def get_conf_topics(conf_path='data/2023/5_iclr.csv_topics.csv', topics=['multimodal']):
    df_conf = pd.read_csv(conf_path)

    save_path = conf_path + '_emerging.xlsx'
    writer = pd.ExcelWriter(save_path, engine="xlsxwriter")

    # if topics within the list of topics
    df_conf_topics = df_conf[df_conf['topic'].str.contains('|'.join(topics), case=False, na=False)].copy()

    ss_citations = []
    for title in tqdm(df_conf_topics['title'], ncols=80):
        ss_citations.append(get_paper_citation(title))
        time.sleep(1)
    
    df_conf_topics['ss_citations'] = ss_citations
    df_conf_topics['google_scholar_link'] = df_conf_topics['title'].apply(lambda x: 'https://scholar.google.com/scholar?q=' + x.replace(' ', '+'))

    df_conf_topics[['title','year','source','ss_citations','google_scholar_link', 'topic']].to_excel(writer, sheet_name='topics', index=False)
    writer.close()
    print(f'Saved to {save_path}')

# 2023

## 2023, ICLR, May

In [3]:
# get_conf_topics_all(conf_path='data/2023/5_iclr.csv_topics.csv', topics=['multimodal', 'diffusion models', 'llm', 'self-supervision'])

## 2023, CVPR, June

In [4]:
# get_conf_topics_all(conf_path='data/2023/6_cvpr.csv_topics.csv', topics=['3d', 'zero_few-shot', 'transformer', 'segmentation', 'multimodal', 
# 'imitation learning', 'radiance field', 'motion', 'diffusion models', 'multi-agent'])

## 2023, ICML, July

In [5]:
# get_conf_topics_all(conf_path='data/2023/7_icml.csv_topics.csv', topics=['zero_few-shot', 'graph', 'optimization', 'reinforcement learning', 'representation',
# 'motion', 'decision trees', 'radiance field', 'diffusion models', 'federated learning'])

## 2023, ICCV, Oct.

In [6]:
# get_conf_topics(conf_path='data/2023/10_ICCV.csv_topics.csv', topics=['object detection', '3d', 'segmentation', 'transformer', 'zero_few-shot'])

## 2023, NIPS, Dec.

In [7]:
# get_conf_topics(conf_path='data/2023/12_nips.csv_topics.csv', topics=['object detection', 'zero_few-shot', 'graph', 'optimization', 'llm', 'radiance field', 'motion', 'differential equation solving', 'rl'])

# 2024

## 2024, ICLR, May

In [8]:
# get_conf_topics(conf_path='data/2024/5_iclr.csv_topics.csv', topics=['graph', 'zero_few-shot', 'llm', 'transformer', 'generative model', 'reward model', 'watermark', 'foundation models', 'model merging', 'diffusion models'])

## 2024, CVPR, June

In [9]:
# get_conf_topics(conf_path='data/2024/6_cvpr.csv_topics.csv', topics=['3d', 'generative model', 'segmentation', 'multimodal', 'zero_few-shot', 'watermark', 'streaming', 'token', 'autonomous driving', 'entangle'])

## 2024, ICML, July

In [11]:
# get_conf_topics(conf_path='data/2024/7_icml.csv_topics.csv', topics=['deformation', 'autonomous driving', 'object detection', 'splatting', 'token', 'graph', 'zero_few-shot', 'llm', 'optimization', 'transformer'])

## 2024, NIPS, Dec.

In [3]:
get_conf_topics(conf_path='data/2024/12_nips.csv_topics.csv', topics=['zero_few-shot', 'graph', 'llm', 'optimization', 'transformer', '4D', 'splatting', 'diffusion transformer', 'state space model', 'text-to-video'])

  0%|▏                                      | 12/3543 [00:40<3:06:20,  3.17s/it]

Error in get citation for GREATS: Online Selection of High Quality Data for LLM Training in Every Iteration


  1%|▎                                      | 27/3543 [01:34<3:20:25,  3.42s/it]

Error in get citation for Learning Noisy Halfspaces with a Margin: Massart is No Harder than Random


  1%|▍                                      | 37/3543 [02:10<3:46:01,  3.87s/it]

...

Error in get citation for DRIP: Unleashing Diffusion Priors for Joint Foreground and Alpha Prediction in Image Matting


100%|████████████████████████████████████▉| 3538/3543 [4:08:29<00:17,  3.53s/it]

Error in get citation for SampDetox: Black box Backdoor Defense via Perturbation based Sample Detoxification


100%|█████████████████████████████████████| 3543/3543 [4:08:46<00:00,  4.21s/it]


Saved to data/2024/12_nips.csv_topics.csv_emerging.xlsx
